In [31]:
import requests
import pandas as pd
import time
import json
import re

In [40]:
API_TOKEN = ''

headers = {
    'Authorization': f'Bearer {API_TOKEN}'
}

response = requests.get('https://lichess.org/api/account', headers=headers)

# if response.status_code == 200:
#     print(response.json())
# else:
#     print(f"Error fetching account data: {response.status_code}")

In [54]:
def get_team_members(team_id, full=False):
    """Fetch all users from a Lichess team."""
    url = f'https://lichess.org/api/team/{team_id}/users'
    params = {'full': str(full).lower()}
    response = requests.get(url)
    
    if response.status_code == 200:
        # Process the NDJSON response
        members = [json.loads(line) for line in response.iter_lines(decode_unicode=True)]
        return members
    else:
        print(f"Error fetching team members: {response.status_code}")
        return []

In [50]:
def get_user_games(username, max_games=100, retries=3):
    """Fetch game history for a specific Lichess user (PNG Format)."""
    url = f'https://lichess.org/api/games/user/{username}?max={max_games}&format=pgn'
    for i in range(retries):
        try:
            response = requests.get(url, headers=headers, timeout=10)  # Add timeout
            if response.status_code == 200:
                return response.text
            elif response.status_code == 429:
                print(f"Rate limit hit for user {username}. Retrying in {2 ** i} seconds...")
                time.sleep(2 ** i)  # Exponential backoff
            else:
                print(f"Error fetching games for user {username}: {response.status_code}")
                return ""
        except requests.exceptions.Timeout:
            print(f"Timeout occurred for user {username}. Retrying in {2 ** i} seconds...")
            time.sleep(2 ** i)
        except requests.exceptions.ConnectionError as e:
            print(f"Connection error for user {username}: {e}")
            return ""
    return ""

In [43]:
import re

def process_pgn_data(pgn_data):
    """Process PGN game data into structured format."""
    games = []
    # Blank line between games
    games_data = pgn_data.strip().split("\n\n")
    
    for game in games_data:
        game_info = {}

        # Re to check if it returns None
        def extract_field(regex, text, default="Unknown"):
            match = re.search(regex, text)
            return match.group(1) if match else default

        # Organize info, else = 'Unknown'
        game_info['event'] = extract_field(r'\[Event "(.*?)"\]', game)
        game_info['site'] = extract_field(r'\[Site "(.*?)"\]', game)
        game_info['date'] = extract_field(r'\[Date "(.*?)"\]', game)
        game_info['white'] = extract_field(r'\[White "(.*?)"\]', game)
        game_info['black'] = extract_field(r'\[Black "(.*?)"\]', game)
        game_info['result'] = extract_field(r'\[Result "(.*?)"\]', game)
        game_info['white_elo'] = extract_field(r'\[WhiteElo "(.*?)"\]', game)
        game_info['black_elo'] = extract_field(r'\[BlackElo "(.*?)"\]', game)
        game_info['time_control'] = extract_field(r'\[TimeControl "(.*?)"\]', game)
        game_info['eco'] = extract_field(r'\[ECO "(.*?)"\]', game)
        game_info['termination'] = extract_field(r'\[Termination "(.*?)"\]', game)

        # Extract all moves (assuming everything after the headers)
        moves = game.split("\n")[-1].strip()
        game_info['moves'] = moves

        games.append(game_info)
    
    return games


In [44]:
def save_to_csv(games, filename='lichess_user_games.csv'):
    """Save the collected game data to a CSV file."""
    df = pd.DataFrame(games)
    df.to_csv(filename, index=False)
    print(f"Saved {len(games)} games to {filename}")

In [55]:
team_id = 'coders'  # Replace with the team id (I'm using coders for now)
members = get_team_members(team_id)

if members:
    all_games = []
    for member in members:
        username = member['id']  # 'id' = username 
        print(f"Fetching games for user: {username}")
        pgn_data = get_user_games(username, max_games=10)  # Limit to 10 games per user for now
        if pgn_data:
            games = process_pgn_data(pgn_data)
            all_games.extend(games)
    save_to_csv(all_games, 'lichess_team_games.csv')
else:
    print("No members found or error retrieving members.")

Fetching games for user: fullberserker
Fetching games for user: aloremipsumog
Fetching games for user: kanjiklub66
Fetching games for user: samanyayghosh
Fetching games for user: pgod1000
Fetching games for user: vivaclover
Fetching games for user: whols42
Fetching games for user: yashwin01
Fetching games for user: emilshmemil
Fetching games for user: cadbury-world-1978
Fetching games for user: antirazh
Rate limit hit for user antirazh. Retrying in 1 seconds...
Fetching games for user: jocoder23
Rate limit hit for user jocoder23. Retrying in 1 seconds...
Fetching games for user: repensar
Rate limit hit for user repensar. Retrying in 1 seconds...
Fetching games for user: totalnoob69
Fetching games for user: jashud
Rate limit hit for user jashud. Retrying in 1 seconds...
Fetching games for user: bruteforceeyes
Rate limit hit for user bruteforceeyes. Retrying in 1 seconds...
Fetching games for user: xopiad
Fetching games for user: sunglases
Rate limit hit for user sunglases. Retrying in 1

Fetching games for user: arminius4
Rate limit hit for user arminius4. Retrying in 1 seconds...
Fetching games for user: javierpelayo
Rate limit hit for user javierpelayo. Retrying in 1 seconds...
Fetching games for user: fluffy_cookie
Fetching games for user: fady_2007
Rate limit hit for user fady_2007. Retrying in 1 seconds...
Fetching games for user: sharkbaithoohahah
Rate limit hit for user sharkbaithoohahah. Retrying in 1 seconds...
Fetching games for user: viratcarlsen
Rate limit hit for user viratcarlsen. Retrying in 1 seconds...
Fetching games for user: nathanlo99
Rate limit hit for user nathanlo99. Retrying in 1 seconds...
Timeout occurred for user nathanlo99. Retrying in 2 seconds...
Timeout occurred for user nathanlo99. Retrying in 4 seconds...
Fetching games for user: asdiapotatisen
Timeout occurred for user asdiapotatisen. Retrying in 1 seconds...
Timeout occurred for user asdiapotatisen. Retrying in 2 seconds...
Timeout occurred for user asdiapotatisen. Retrying in 4 secon

Timeout occurred for user firstcamel. Retrying in 2 seconds...
Timeout occurred for user firstcamel. Retrying in 4 seconds...
Fetching games for user: variantsqueen
Timeout occurred for user variantsqueen. Retrying in 1 seconds...
Timeout occurred for user variantsqueen. Retrying in 2 seconds...
Timeout occurred for user variantsqueen. Retrying in 4 seconds...
Fetching games for user: alanliang
Timeout occurred for user alanliang. Retrying in 1 seconds...
Timeout occurred for user alanliang. Retrying in 2 seconds...
Timeout occurred for user alanliang. Retrying in 4 seconds...
Fetching games for user: harirakul
Timeout occurred for user harirakul. Retrying in 1 seconds...
Timeout occurred for user harirakul. Retrying in 2 seconds...
Timeout occurred for user harirakul. Retrying in 4 seconds...
Fetching games for user: shawnbadger
Timeout occurred for user shawnbadger. Retrying in 1 seconds...
Timeout occurred for user shawnbadger. Retrying in 2 seconds...
Timeout occurred for user shaw

Fetching games for user: hatterix
Fetching games for user: jamierrobillardsr
Rate limit hit for user jamierrobillardsr. Retrying in 1 seconds...
Fetching games for user: himanscience
Rate limit hit for user himanscience. Retrying in 1 seconds...
Fetching games for user: thomasmarvoloriddle
Rate limit hit for user thomasmarvoloriddle. Retrying in 1 seconds...
Fetching games for user: nanoman6666
Rate limit hit for user nanoman6666. Retrying in 1 seconds...
Fetching games for user: foowalksintoabar
Rate limit hit for user foowalksintoabar. Retrying in 1 seconds...
Fetching games for user: dothemath
Fetching games for user: dadroid
Rate limit hit for user dadroid. Retrying in 1 seconds...
Fetching games for user: frazjp65
Rate limit hit for user frazjp65. Retrying in 1 seconds...
Timeout occurred for user frazjp65. Retrying in 2 seconds...
Timeout occurred for user frazjp65. Retrying in 4 seconds...
Fetching games for user: operaphantom
Timeout occurred for user operaphantom. Retrying in 

Timeout occurred for user somebodyelse. Retrying in 4 seconds...
Fetching games for user: gevanni
Timeout occurred for user gevanni. Retrying in 1 seconds...
Timeout occurred for user gevanni. Retrying in 2 seconds...
Timeout occurred for user gevanni. Retrying in 4 seconds...
Fetching games for user: kennyschess
Timeout occurred for user kennyschess. Retrying in 1 seconds...
Timeout occurred for user kennyschess. Retrying in 2 seconds...
Timeout occurred for user kennyschess. Retrying in 4 seconds...
Fetching games for user: xadrez_pt
Timeout occurred for user xadrez_pt. Retrying in 1 seconds...
Timeout occurred for user xadrez_pt. Retrying in 2 seconds...
Timeout occurred for user xadrez_pt. Retrying in 4 seconds...
Fetching games for user: th1sissparta
Fetching games for user: grandmaster60
Fetching games for user: soaringwombat
Fetching games for user: alfredzkinston
Fetching games for user: eeeeeta
Fetching games for user: noobz
Fetching games for user: tangmc
Fetching games for u

Fetching games for user: kennykena
Rate limit hit for user kennykena. Retrying in 1 seconds...
Fetching games for user: block
Fetching games for user: jacob135135
Rate limit hit for user jacob135135. Retrying in 1 seconds...
Fetching games for user: 1000let
Rate limit hit for user 1000let. Retrying in 1 seconds...
Error fetching games for user 1000let: 404
Fetching games for user: vitus
Rate limit hit for user vitus. Retrying in 1 seconds...
Fetching games for user: ivgl
Rate limit hit for user ivgl. Retrying in 1 seconds...
Fetching games for user: zotov228
Fetching games for user: yashar_sb_sb
Rate limit hit for user yashar_sb_sb. Retrying in 1 seconds...
Fetching games for user: 0day
Rate limit hit for user 0day. Retrying in 1 seconds...
Timeout occurred for user 0day. Retrying in 2 seconds...
Timeout occurred for user 0day. Retrying in 4 seconds...
Fetching games for user: hellball
Timeout occurred for user hellball. Retrying in 1 seconds...
Timeout occurred for user hellball. Retr